In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
DATASET_LOCATION = '/content/drive/MyDrive/ai-playground/ra-text-classification-spectrogram/spectrograms/bert-imdb-3000-categorized'
data_dir = pathlib.Path(DATASET_LOCATION)

In [4]:
TRAIN_FRACTION = 0.8
batch_size = 32 # default batch size of data
img_height = 224 # default height to which image is resized
img_width = 224 # default width to which image is resized
seed = 2022 # random seed for shuffling and transformations

In [5]:
ds_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "training",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 3466 files belonging to 2 classes.
Using 2773 files for training.


In [6]:
ds_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "validation",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 3466 files belonging to 2 classes.
Using 693 files for validation.


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
normalization_layer = layers.Rescaling(1./255)

In [9]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [10]:
class_names = ds_train.class_names
num_classes = len(class_names)
print(class_names)

['neg', 'pos']


In [11]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [13]:
epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/100
87/87 [==============================] - 56s 556ms/step - loss: 0.7126 - accuracy: 0.5380 - val_loss: 0.6916 - val_accuracy: 0.5281
Epoch 2/100
87/87 [==============================] - 3s 36ms/step - loss: 0.6894 - accuracy: 0.5499 - val_loss: 0.6899 - val_accuracy: 0.5281
Epoch 3/100
87/87 [==============================] - 3s 35ms/step - loss: 0.6852 - accuracy: 0.5499 - val_loss: 0.6817 - val_accuracy: 0.5281
Epoch 4/100
87/87 [==============================] - 3s 35ms/step - loss: 0.6687 - accuracy: 0.5929 - val_loss: 0.6783 - val_accuracy: 0.5859
Epoch 5/100
87/87 [==============================] - 3s 36ms/step - loss: 0.6482 - accuracy: 0.6217 - val_loss: 0.6518 - val_accuracy: 0.6032
Epoch 6/100
87/87 [==============================] - 3s 35ms/step - loss: 0.6296 - accuracy: 0.6466 - val_loss: 0.6515 - val_accuracy: 0.6162
Epoch 7/100
87/87 [==============================] - 3s 35ms/step - loss: 0.6255 - accuracy: 0.6545 - val_loss: 0.6230 - val_accuracy: 0.6349
Epoc

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()